In [1]:
import requests
import pandas as pd
import json

In [13]:
def get_network_info(id=None):
    if id:
        id = f"id={','.join([str(i) for i in id])}&"
    else:
        id = ""

    url = f"https://api.synopticdata.com/v2/networks?{id}token=d25c2abe02b94001a82e7790d9c30f06"
    data = json.loads(requests.get(url).text)

    df = pd.DataFrame(data["MNET"]).set_index("ID")
    df.index = df.index.astype(int)
    return df


networks = get_network_info([1])
networks

,CATEGORY,REPORTING_STATIONS,TOTAL_RESTRICTED,ACTIVE_RESTRICTED,LAST_OBSERVATION,URL,PERCENT_REPORTING,PERIOD_CHECKED,TOTAL_STATIONS,ACTIVE_STATIONS,PERIOD_OF_RECORD,LONGNAME,SHORTNAME,PERCENT_ACTIVE
ID,,,,,,,,,,,,,,
1,4,2504,0,0,2023-07-10T13:35:00Z,None,96.23,120,3524,2602,"{'start': '1997-01-01T00:00:00Z', 'end': '2023...",ASOS/AWOS,ASOS/AWOS,73.84


In [2]:
station_selector = "UKBKB,WBB"
units = "english"
obtimezone = "local"
token = "d25c2abe02b94001a82e7790d9c30f06"

base_url = "https://api.synopticdata.com/v2/stations/latest?"
arguments = [
    f"stid={station_selector}",
    f"units={units}",
    f"obtimezone={obtimezone}",
    f"token={token}",
    f"within=60",
]
url = base_url + "&".join(arguments)
url

'https://api.synopticdata.com/v2/stations/latest?stid=UKBKB,WBB&units=english&obtimezone=local&token=d25c2abe02b94001a82e7790d9c30f06&within=60'

In [39]:
data = json.loads(requests.get(url).text)
data

{'UNITS': {'wind_speed': 'knots',
  'air_temp': 'Fahrenheit',
  'wind_chill': 'Fahrenheit',
  'solar_radiation': 'W/m**2',
  'wind_gust': 'knots',
  'pressure': 'Millibars',
  'altimeter': 'INHG',
  'wind_direction': 'Degrees',
  'relative_humidity': '%',
  'ozone_concentration': 'ppb',
  'sea_level_pressure': 'Millibars',
  'elevation': 'ft',
  'volt': 'volts',
  'dew_point_temperature': 'Fahrenheit',
  'heat_index': 'Fahrenheit',
  'precip_accum_24_hour': 'Inches',
  'precip_accum_since_local_midnight': 'Inches',
  'position': 'ft',
  'precip_accum_one_minute': 'Inches',
  'wind_cardinal_direction': 'code'},
 'QC_SUMMARY': {'QC_SHORTNAMES': {'1': 'sl_range_check'},
  'QC_CHECKS_APPLIED': ['sl_range_check'],
  'PERCENT_OF_TOTAL_OBSERVATIONS_FLAGGED': 0.0,
  'QC_SOURCENAMES': {'1': 'SynopticLabs'},
  'TOTAL_OBSERVATIONS_FLAGGED': 0.0,
  'QC_NAMES': {'1': 'SynopticLabs Range Check'}},
 'STATION': [{'STATUS': 'ACTIVE',
   'MNET_ID': '153',
   'PERIOD_OF_RECORD': {'start': '1997-01-01T00:

In [40]:
data["STATION"][0].keys()

dict_keys(['STATUS', 'MNET_ID', 'PERIOD_OF_RECORD', 'ELEVATION', 'NAME', 'STID', 'SENSOR_VARIABLES', 'ELEV_DEM', 'LONGITUDE', 'UNITS', 'STATE', 'OBSERVATIONS', 'RESTRICTED', 'QC_FLAGGED', 'LATITUDE', 'TIMEZONE', 'ID'])

In [42]:
data = json.loads(requests.get(url).text)

Z = {}
for stn in data["STATION"]:
    obs = stn.pop("OBSERVATIONS")
    senvars = stn.pop("SENSOR_VARIABLES")

    metadata_table = pd.DataFrame(stn).T.sort_index()

    value_table = pd.DataFrame(obs).T.sort_index()
    value_table["unit"] = [
        data["UNITS"]["_".join(i.split("_")[:-2])] for i in value_table.index
    ]
    value_table = value_table[["value", "unit", "date_time"]]
    value_table["date_time"] = pd.to_datetime(value_table["date_time"])
    value_table

    Z[stn["STID"]] = (metadata_table, value_table)

In [41]:
Z["WBB"][0]["start"]["NAME"]

'U of U William Browning Building'

In [6]:
data["UNITS"]

{'wind_speed': 'knots',
 'air_temp': 'Fahrenheit',
 'wind_chill': 'Fahrenheit',
 'solar_radiation': 'W/m**2',
 'wind_gust': 'knots',
 'pressure': 'Millibars',
 'altimeter': 'INHG',
 'wind_direction': 'Degrees',
 'relative_humidity': '%',
 'ozone_concentration': 'ppb',
 'sea_level_pressure': 'Millibars',
 'elevation': 'ft',
 'volt': 'volts',
 'dew_point_temperature': 'Fahrenheit',
 'heat_index': 'Fahrenheit',
 'precip_accum_24_hour': 'Inches',
 'precip_accum_since_local_midnight': 'Inches',
 'position': 'ft',
 'precip_accum_one_minute': 'Inches',
 'wind_cardinal_direction': 'code'}

In [7]:
data["STATION"][0].keys()

dict_keys(['STATUS', 'MNET_ID', 'PERIOD_OF_RECORD', 'ELEVATION', 'NAME', 'STID', 'ELEV_DEM', 'LONGITUDE', 'UNITS', 'STATE', 'RESTRICTED', 'QC_FLAGGED', 'LATITUDE', 'TIMEZONE', 'ID'])

In [9]:
# Function to convert timedelta to human-readable format
def format_timedelta(td):
    minutes = td.total_seconds() // 60
    if minutes < 60:
        return f"{int(minutes)} minutes ago"
    hours = minutes // 60
    if hours < 24:
        return f"{int(hours)} hours ago"
    days = hours // 24
    return f"{int(days)} days ago"


value_table["time_ago"] = (pd.Timestamp.utcnow() - value_table["date_time"]).apply(
    format_timedelta
)

In [11]:
120.4 // 60

2.0

In [10]:
value_table

,value,unit,date_time,time_ago
air_temp_value_1,69.0,Fahrenheit,2023-07-10 07:00:00-06:00,9 minutes ago
altimeter_value_1,29.56,INHG,2023-07-10 07:00:00-06:00,9 minutes ago
dew_point_temperature_value_1d,42.24,Fahrenheit,2023-07-10 07:00:00-06:00,9 minutes ago
precip_accum_24_hour_value_1,0.0,Inches,2023-07-10 07:00:00-06:00,9 minutes ago
precip_accum_since_local_midnight_value_1,0.0,Inches,2023-07-10 07:00:00-06:00,9 minutes ago
pressure_value_1d,841.13,Millibars,2023-07-10 07:00:00-06:00,9 minutes ago
relative_humidity_value_1,38.0,%,2023-07-10 07:00:00-06:00,9 minutes ago
sea_level_pressure_value_1d,992.22,Millibars,2023-07-10 07:00:00-06:00,9 minutes ago
wind_cardinal_direction_value_1d,NW,code,2023-07-10 07:00:00-06:00,9 minutes ago
wind_direction_value_1,326.0,Degrees,2023-07-10 07:00:00-06:00,9 minutes ago


In [60]:
Z["WBB"][0]
states = {df["start"].get("STATE") for _, (df, _) in Z.items()}
states

{'UT'}

In [61]:
[i["start"]["LATITUDE"] for i, _ in Z.values()]

['40.76623', '40.09867']

In [14]:
network_ids = {df["start"]["MNET_ID"] for stid, (df, _) in Z.items()}
network_df = get_network_info(network_ids)

network_ids

{'153', '65'}

In [15]:
network_df

,CATEGORY,REPORTING_STATIONS,TOTAL_RESTRICTED,ACTIVE_RESTRICTED,LAST_OBSERVATION,URL,PERCENT_REPORTING,PERIOD_CHECKED,TOTAL_STATIONS,ACTIVE_STATIONS,PERIOD_OF_RECORD,LONGNAME,SHORTNAME,PERCENT_ACTIVE
ID,,,,,,,,,,,,,,
65,8,12453,0,0,2023-07-10T13:12:00Z,None,94.21,120,28536,13218,"{'start': '2002-06-20T00:00:00Z', 'end': '2023...",Automatic Position Reporting System WX NET/Cit...,APRSWXNET/CWOP,46.32
153,6,57,0,0,2023-07-10T14:00:00Z,None,91.94,120,87,62,"{'start': '1997-01-01T00:00:00Z', 'end': '2023...",University of Utah MesoWest Mesonet,UUNET,71.26


In [18]:
metadata_table

,start,end,position,elevation
ELEVATION,4734,4734,4734,4734
ELEV_DEM,4740.8,4740.8,4740.8,4740.8
ID,37032,37032,37032,37032
LATITUDE,40.09867,40.09867,40.09867,40.09867
LONGITUDE,-111.62767,-111.62767,-111.62767,-111.62767
MNET_ID,65,65,65,65
NAME,EW2355 Spanish Fork,EW2355 Spanish Fork,EW2355 Spanish Fork,EW2355 Spanish Fork
PERIOD_OF_RECORD,2013-03-13T00:00:00Z,2023-07-10T12:30:00Z,NaN,NaN
QC_FLAGGED,False,False,False,False
RESTRICTED,False,False,False,False


In [32]:
metadata_table = metadata_table.reindex(metadata_table.index.insert(6, "MNET_NAME"))
metadata_table.loc["MNET_NAME"] = network_df.loc[
    int(metadata_table["end"]["MNET_ID"])
].LONGNAME
metadata_table

,start,end,position,elevation
ELEVATION,4734,4734,4734,4734
ELEV_DEM,4740.8,4740.8,4740.8,4740.8
ID,37032,37032,37032,37032
LATITUDE,40.09867,40.09867,40.09867,40.09867
LONGITUDE,-111.62767,-111.62767,-111.62767,-111.62767
MNET_ID,65,65,65,65
MNET_NAME,Automatic Position Reporting System WX NET/Cit...,Automatic Position Reporting System WX NET/Cit...,Automatic Position Reporting System WX NET/Cit...,Automatic Position Reporting System WX NET/Cit...
NAME,EW2355 Spanish Fork,EW2355 Spanish Fork,EW2355 Spanish Fork,EW2355 Spanish Fork
PERIOD_OF_RECORD,2013-03-13T00:00:00Z,2023-07-10T12:45:00Z,NaN,NaN
QC_FLAGGED,False,False,False,False


In [28]:
metadata_table.reindex(metadata_table.index.insert(4, "hi there"))

,start,end,position,elevation
ELEVATION,4734,4734,4734,4734
ELEV_DEM,4740.8,4740.8,4740.8,4740.8
ID,37032,37032,37032,37032
LATITUDE,40.09867,40.09867,40.09867,40.09867
hi there,NaN,NaN,NaN,NaN
LONGITUDE,-111.62767,-111.62767,-111.62767,-111.62767
MNET_ID,65,65,65,65
NAME,EW2355 Spanish Fork,EW2355 Spanish Fork,EW2355 Spanish Fork,EW2355 Spanish Fork
PERIOD_OF_RECORD,2013-03-13T00:00:00Z,2023-07-10T12:30:00Z,NaN,NaN
QC_FLAGGED,False,False,False,False


In [43]:
network_ids = {df["start"]["MNET_ID"] for stid, (df, _) in Z.items()}
network_df = get_network_info(network_ids)
for stid, (metadata_df, value_df) in Z.items():
    mnet_id = int(metadata_df["end"]["MNET_ID"])
    metadata_df.loc["MNET_ID"] = (
        f"{mnet_id} - {network_df.loc[mnet_id].SHORTNAME} :: {network_df.loc[mnet_id].LONGNAME}"
    )
    Z[stid] = (metadata_df, value_df)

In [44]:
metadata_df

,start,end,position,elevation
ELEVATION,4734,4734,4734,4734
ELEV_DEM,4740.8,4740.8,4740.8,4740.8
ID,37032,37032,37032,37032
LATITUDE,40.09867,40.09867,40.09867,40.09867
LONGITUDE,-111.62767,-111.62767,-111.62767,-111.62767
MNET_ID,65 - APRSWXNET/CWOP :: Automatic Position Repo...,65 - APRSWXNET/CWOP :: Automatic Position Repo...,65 - APRSWXNET/CWOP :: Automatic Position Repo...,65 - APRSWXNET/CWOP :: Automatic Position Repo...
NAME,EW2355 Spanish Fork,EW2355 Spanish Fork,EW2355 Spanish Fork,EW2355 Spanish Fork
PERIOD_OF_RECORD,2013-03-13T00:00:00Z,2023-07-10T13:15:00Z,NaN,NaN
QC_FLAGGED,False,False,False,False
RESTRICTED,False,False,False,False
